In [1]:
!pip install pdf2image
!apt-get install poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.10 [186 kB]
Fetched 186 kB in 0s (452 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126374 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.10_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.10) ...
Setting up poppler-utils (22.02.0-2ubuntu0.10) ...
Processing triggers for man-db (2.10.2-1) ...


In [7]:
#use this code goood

import re
import requests
import io
from pdf2image import convert_from_bytes
from google.colab import files
from IPython.display import HTML, display

def generate_digest_post_for_blogger(drive_url, page_number, digest_title):
    # Extract file ID from Google Drive URL
    if "id=" in drive_url:
        file_id = drive_url.split("id=")[-1]
    elif "file/d/" in drive_url:
        file_id = drive_url.split("file/d/")[1].split("/")[0]
    else:
        raise ValueError("Invalid Google Drive link")

    # Generate download and preview URLs
    download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
    preview_url = f"https://drive.google.com/file/d/{file_id}/preview"

    # Download PDF in memory
    response = requests.get(download_url)
    if response.status_code != 200:
        raise Exception("Failed to fetch PDF from Google Drive")
    pdf_bytes = io.BytesIO(response.content)

    # Use digest_title for image filename
    image_filename = digest_title.replace(" ", "_").lower() + f"_page_{page_number}.png"

    # Convert specified page to PNG
    images = convert_from_bytes(pdf_bytes.read(), first_page=page_number, last_page=page_number)
    images[0].save(image_filename, "PNG")
    files.download(image_filename)

    # Extract digest name and month-year from title
    match = re.match(r"(.+?)\s+(\w+)\s+(\d{4})", digest_title)
    if match:
        digest_name = match.group(1)           # e.g. Shuaa Digest
        month = match.group(2)                # e.g. January
        year = match.group(3)                 # e.g. 2023
        edition = f"{month} {year}"           # e.g. January 2023
    else:
        digest_name = digest_title
        month = "Unknown"
        year = "Unknown"
        edition = "Unknown"

    # Generate labels (tags)
    labels = [
        f"Digests {year}",
        f"{digest_name}s",
        f"{digest_name}s {year}",
        f"{year} {month} Digests",
        f"{month} Digests {year}"

    ]

    # Generate SEO-friendly description (max 150 characters)
    description = f"Read {digest_title} online with Urdu short stories, novels & articles. Free PDF download of latest Urdu digest for readers."
    description = description[:150]

    # Generate HTML post with inline CSS for Blogger
    html_content = f"""<!--Post Title-->
<h1 style="text-align: center;">{digest_title} – Read Online &amp; Download PDF</h1>
<div>
    <p><span style="font-family: Arial, sans-serif;"><i>The new edition of&nbsp;<b>{digest_title}</b>&nbsp;has been published for Urdu readers. This digest includes short stories, novels, and articles. If you are a fan of Urdu digests, you will surely enjoy this edition.</i></span></p>
    <p><i><span style="font-family: Arial, sans-serif;"><br /></span></i></p>
    <span style="font-family: Arial, sans-serif;"><!--Digest Details--></span>
    <h2><i><span style="font-family: Arial, sans-serif;">Digest Details</span></i></h2>
    <ul>
        <li><i><span style="font-family: Arial, sans-serif;"><b>Digest Name:</b>&nbsp;{digest_name}</span></i></li>
        <li><i><span style="font-family: Arial, sans-serif;"><b>Edition:</b>&nbsp;{edition}</span></i></li>
        <li><i><span style="font-family: Arial, sans-serif;"><b>Language:</b>&nbsp;Urdu</span></i></li>
        <li><i><span style="font-family: Arial, sans-serif;"><b>Format:</b>&nbsp;PDF + Read Online</span></i></li>
    </ul>
    <div><i><span style="font-family: Arial, sans-serif;"><br /></span></i></div>
    <span style="font-family: Arial, sans-serif;"><!--Read Online--></span>
    <h2><i><span style="font-family: Arial, sans-serif;">Read Online {digest_title}</span></i></h2>
</div>
<div>
    <div style="display: flex; justify-content: center;">
        <iframe loading="lazy" style="width: 100%; max-width: 640px; height: 480px;" class="digest-iframe" src="{preview_url}"></iframe>
    </div>
    <div style="display: flex; justify-content: center;"><br /></div>
    <div style="display: flex; justify-content: center;">
        <b><span style="color: #ffa400; font-size: x-large;"><br /></span></b>
        <b><span style="color: #ffa400; font-size: x-large;"><br /></span></b>
        <b><span style="color: #ffa400; font-size: x-large;">OR</span></b>
        <b><span style="color: #ffa400; font-size: x-large;"><br /></span></b>
        <b><span style="color: #ffa400; font-size: x-large;"><br /></span></b>
        <br />
    </div>
    <div style="display: flex; justify-content: center;">
        <a href="{download_url}" class="custom-download-btn">👉&nbsp;<i>Download {digest_title} PDF</i></a>
    </div>
    <div style="display: flex; justify-content: center;"><br /></div>
    <div style="display: flex; justify-content: center;">
        <h2 style="font-family: &quot;Times New Roman&quot;, serif; font-style: normal; text-align: left;"><i><span style="font-family: Arial, sans-serif; font-weight: normal;">Tags:</span></i></h2>
    </div>
    <div style="display: flex; justify-content: center;">
        <i>{digest_title} PDF Free Download, Read Online {digest_name}, Latest Urdu Digest {edition}, Free Urdu Novels Digest, Monthly {digest_name}</i>
    </div>
</div>
<div><br /></div>
<div><br /></div>
"""

    # Display HTML code, labels, and description in textarea with copy buttons
    display(HTML(f"""
    <h3>📄 Ready-made Embed Code for Blogger</h3>
    <textarea id="final_code" style="width:100%;height:300px;">{html_content}</textarea>
    <button onclick="navigator.clipboard.writeText(document.getElementById('final_code').value)">📋 Copy Embed Code</button>

    <h3>🏷️ Generated Labels (Tags)</h3>
    <textarea id="labels_code" style="width:100%;height:120px;">{', '.join(labels)}</textarea>
    <button onclick="navigator.clipboard.writeText(document.getElementById('labels_code').value)">📋 Copy Labels</button>

    <h3>📝 SEO-Friendly Description (Max 150 chars)</h3>
    <textarea id="description_code" style="width:100%;height:100px;">{description}</textarea>
    <button onclick="navigator.clipboard.writeText(document.getElementById('description_code').value)">📋 Copy Description</button>

    <p><b>Note:</b> Add the following CSS to your Blogger template's <head> section for the button styling:</p>
    <pre>
&lt;style&gt;
.custom-download-btn {{
    background: linear-gradient(135deg, #e63946, #ff6b6b);
    border-radius: 8px;
    color: white;
    font-size: 16px;
    font-weight: bold;
    padding: 12px 24px;
    text-align: center;
    text-decoration: none;
    display: inline-flex;
    align-items: center;
    gap: 8px;
    transition: background 0.3s ease;
    box-shadow: none;
}}
.custom-download-btn:hover {{
    background: linear-gradient(135deg, #ff6b6b, #e63946);
}}
&lt;/style&gt;
    </pre>
    """))

# Example usage
drive_link = "https://drive.google.com/file/d/10yOnCa2yZVnYNJC_o3JzLUwT358CQ4sn/view?usp=drive_link"
page_number = 1
digest_title = "Suspense Digest August 2025"
generate_digest_post_for_blogger(drive_link, page_number, digest_title)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

preview only


In [ ]:
from IPython.display import HTML, display

def generate_minimal_digest_post(drive_url):
    # Extract file ID from Google Drive URL
    if "id=" in drive_url:
        file_id = drive_url.split("id=")[-1]
    elif "file/d/" in drive_url:
        file_id = drive_url.split("file/d/")[1].split("/")[0]
    else:
        raise ValueError("Invalid Google Drive link")

    # Generate download and preview URLs
    download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
    preview_url = f"https://drive.google.com/file/d/{file_id}/preview"

    # Generate HTML post for Blogger
    html_content = f"""<div><br /></div>
<div><br /></div>
<div style="display: flex; justify-content: center;">
    <iframe loading="lazy" style="width: 100%; max-width: 640px; height: 480px;" class="digest-iframe" src="{preview_url}"></iframe>
</div>
<div><br /></div>
<div style="display: flex; justify-content: center;">
    <a href="{download_url}" class="custom-download-btn">Download</a>
</div>
<div><br /></div>
<div><br /></div>
<div><br /></div>
<div><br /></div>
"""

    # Display HTML code in textarea with copy button
    display(HTML(f"""
    <h3>📄 Ready-made Embed Code for Blogger</h3>
    <textarea id="final_code" style="width:100%;height:200px;">{html_content}</textarea>
    <button onclick="navigator.clipboard.writeText(document.getElementById('final_code').value)">📋 Copy Embed Code</button>
    <p><b>Note:</b> Add the following CSS to your Blogger template's <head> section for the button styling:</p>
    <pre>
&lt;style&gt;
.custom-download-btn {{
    background: linear-gradient(135deg, #d32f2f, #ff6659);
    border-radius: 10px;
    color: white;
    font-size: 18px;
    font-weight: bold;
    padding: 15px 30px;
    text-align: center;
    text-decoration: none;
    box-shadow: 0 2px 8px rgba(0,0,0,0.15);
    transition: all 0.3s ease;
    display: inline-block;
}}
.custom-download-btn:hover {{
    background: linear-gradient(135deg, #ff6659, #d32f2f);
    box-shadow: 0 4px 12px rgba(0,0,0,0.25);
}}
&lt;/style&gt;
    </pre>
    """))

# Example usage
drive_link = "https://drive.google.com/file/d/1r8XCtfu96TTejPv50LPss48OO2_d7KvA/view?usp=drive_link"
generate_minimal_digest_post(drive_link)